In [46]:
import os 
import sys 
import pandas as pd 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [47]:
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [48]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up 

In [50]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [51]:
os.environ.update(credentials)

In [52]:
scope = "user-read-recently-played"

In [53]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

## Functions

In [54]:
def split_utc_time_str(time_str):
    part1 = datetime.datetime.strptime(temp1, '%Y-%m-%dT%H:%M:%S.%fZ') - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [55]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after=after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    
    if recently_played['cursors'] == None:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['after_ts'] = after
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
            
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name'] 
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']

        if song['context'] != None:
            temp_song_dict['playlist_id'] = song['context']['uri']
        else:
            temp_song_dict['playlist_id'] = None
        songs_list.append(temp_song_dict)

    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id', 'after_ts']]
    
    return(out_series)

## Get time of last Spotify Pull

In [56]:
db_location = 'data/listening_history.db'

In [57]:
con = sqlite3.connect(db_location)

In [58]:
cursor = con.cursor()

In [59]:
try:
    rslt = cursor.execute('select max(after_ts) from Listening_History').fetchone()
    latest_time_pull = rslt[0]
    print("Latest pull from ", 
    datetime.datetime.fromtimestamp(int(latest_time_pull)/1000)) 
except Exception as e:
    # Mainly for first run through when table doesn't exist yet 
    ## Will take the latest recent history available 
    print(e)
    latest_time_pull = None

Latest pull from  2020-10-26 01:13:23.195000


## Pull from Spotify and Store to DB 

In [66]:
newly_played = get_recently_played(latest_time_pull)

Getting songs at  2020-11-04 04:40:18.176130
There are 50 songs between  2020-11-03 17:54:25.175000   and   2020-11-03 23:49:13.113000


In [67]:
newly_played.head()

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,F2020,Avenue Beat,2020-11-03,13:54,3.030850,75,spotify:track:7opTGnWVJcm1e5mWWOldpg,spotify:artist:1htYtUdXVMwR6HCSs821Lc,None,1604447353113
1,Conversations in the Dark - John Legend vs. Da...,John Legend,2020-11-03,13:54,3.503933,74,spotify:track:20d27F17AZOxTJOEHAVK2Y,spotify:artist:5y2Xq6xcjJb2jVM54GHK3t,None,1604447353113
2,Where Am I At,MAX,2020-11-03,13:54,3.492500,54,spotify:track:0aZAIRksvTw99b2fQ6w1VG,spotify:artist:1bqxdqvUtPWZri43cKHac8,None,1604447353113
3,Confidant,BabyJake,2020-11-03,13:54,3.043333,59,spotify:track:09PziVJjD74IagSfdN58wH,spotify:artist:07Asx51VtMw5kbNXKrpZlq,None,1604447353113
4,I Should Probably Go To Bed,Dan + Shay,2020-11-03,13:54,2.863767,78,spotify:track:5ovVcYo2MvjVydFwFyaaqy,spotify:artist:7z5WFjZAIYejWy0NI5lv4T,None,1604447353113


In [68]:
con = sqlite3.connect(db_location)
newly_played.to_sql('Listening_History', con, if_exists='append' )

In [69]:
con.commit()
con.close()